In [1]:
!pip3 install grid2op[optional]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 34.2 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 43.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 58.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.8/853.8 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 5.2 MB/s eta 0:00:00
  Created wheel for pandapower: filename=pandapower-2.14.11-py3-none-any.whl size=13131025 sha256=40982da3d6d20502fece46ab9af828321e5cbdeba3eeeb150dd497c581939e0f
  Stored in directory: /root/.cache/pip/wheels/7f/de/5a/7b00f385eb06d1fb1f7c1cd06f9bb901709c038d3899548cf1
  Created wheel for pygifsicle: filename=pygifsicle-1.1.0-py3

In [2]:
!git clone https://github.com/grid2op/Grid2Op.git
!cd Grid2Op
!pip3 install -U .
!cd ..

Cloning into 'Grid2Op'...
remote: Enumerating objects: 29036, done.
remote: Counting objects: 100% (5811/5811), done.
remote: Compressing objects: 100% (1826/1826), done.
remote: Total 29036 (delta 4011), reused 5279 (delta 3910), pack-reused 23225 (from 1)
Receiving objects: 100% (29036/29036), 87.82 MiB | 37.07 MiB/s, done.
Resolving deltas: 100% (21120/21120), done.
ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


## PPO agent interacting in environment 

In [4]:
!pip install l2rpn_baselines

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.8/214.8 kB 4.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for l2rpn_baselines: filename=l2rpn_baselines-0.8.0-py3-none-any.whl size=315439 sha256=ccd150c4b7a64fc8f7ba000ecfbec21a4cb0814f4d32334347c2ec99ef425ef4
  Stored in directory: /root/.cache/pip/wheels/a0/96/c0/3819e51b210c41219b07309d77b9bf92e67932026b9a8c9fd9
Successfully built l2rpn_baselines


In [32]:
import gymnasium as gym

import grid2op
import copy
from typing import Dict, Literal, Any, OrderedDict, Optional, Tuple, Union
from grid2op import gym_compat
from grid2op.Parameters import Parameters
from grid2op.Action import PlayableAction
from grid2op.Observation import CompleteObservation
from grid2op.Reward import L2RPNReward, N1Reward, CombinedScaledReward
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from gymnasium.spaces import Discrete, MultiDiscrete, Box
from grid2op.gym_compat import GymEnv, BoxGymObsSpace, DiscreteActSpace, BoxGymActSpace, MultiDiscreteActSpace, ContinuousToDiscreteConverter
from gymnasium import spaces
from lightsim2grid import LightSimBackend


In [ ]:
from gymnasium import spaces

In [6]:
import gym
from stable_baselines3 import DQN
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecVideoRecorder
import wandb
from wandb.integration.sb3 import WandbCallback


In [ ]:
from stable_baselines3.common.buffers import ReplayBuffer
from grid2op.Reward import LinesCapacityReward, LinesReconnectedReward, CloseToOverflowReward , AlertReward # or any other rewards
from lightsim2grid import LightSimBackend  # highly recommended !
from grid2op.Chronics import MultifolderWithCache  # highly recommended for training
from stable_baselines3.common.buffers import ReplayBuffer
from l2rpn_baselines.DoubleDuelingDQN.prioritized_replay_buffer import PrioritizedReplayBuffer
from stable_baselines3 import DQN, HerReplayBuffer
from stable_baselines3.her.goal_selection_strategy import GoalSelectionStrategy # 'final', 'episode', 'future'
import numpy as np


In [81]:
# Gymnasium environment wrapper around Grid2Op environment
class Gym2OpEnvWrapper(gym.Env):
    def __init__(
            self,env_config: Dict[Literal["backend_cls",
                                          "backend_options",
                                          "env_name",
                                          "env_is_test",
                                          "obs_attr_to_keep",
                                          "act_type",
                                          "act_attr_to_keep"],
                                  Any] = None,reward_class=CombinedScaledReward, other_rewards = {}, training=True, baseline=True):
        super().__init__()
        if env_config is None:
            self.env_config = {}
        else:
            self.env_config=env_config
            
        backend_cls = LightSimBackend
        if "backend_cls" in self.env_config:
            backend_cls = self.env_config["backend_cls"]
        backend_options = {}
        if "backend_options" in self.env_config:
            backend_options = self.env_config["backend_options"]
        self._backend = backend_cls(**backend_options)

        #self._backend = LightSimBackend()
        self._env_name = "l2rpn_case14_sandbox"  # DO NOT CHANGE
        self.baseline = baseline
        action_class = PlayableAction
        observation_class = CompleteObservation
        self.reward_class = reward_class # Setup further below
        self.other_rewards = other_rewards

        # DO NOT CHANGE Parameters
        # See https://grid2op.readthedocs.io/en/latest/parameters.html
        p = Parameters()
        p.MAX_SUB_CHANGED = 4  # Up to 4 substations can be reconfigured each timestep
        p.MAX_LINE_STATUS_CHANGED = 4  # Up to 4 powerline statuses can be changed each timestep
        
        if training:
            # Make grid2op env
            self._g2op_env = grid2op.make(
                self._env_name, backend=self._backend, test=False,
                action_class=action_class, observation_class=observation_class,
                reward_class=self.reward_class, param=p, other_rewards=self.other_rewards 
            )

        ##########
        # REWARD #
        ##########
        # NOTE: This reward should not be modified when evaluating RL agent
        # See https://grid2op.readthedocs.io/en/latest/reward.html
        else:
            self._g2op_env = grid2op.make(
                self._env_name, backend=self._backend, test=False,
                action_class=action_class, observation_class=observation_class,
                reward_class=self.reward_class, param=p
            )
            
        cr = self._g2op_env.get_reward_instance()
        cr.addReward("N1", N1Reward(), 1.0)
        cr.addReward("L2RPN", L2RPNReward(), 1.0)
        #reward = N1 + L2RPN
        cr.initialize(self._g2op_env)
        ##########

        self._gym_env = gym_compat.GymEnv(self._g2op_env)

        self.setup_observations()
        self.setup_actions()

        self.observation_space = self.setup_observations()
        self.action_space = self.setup_actions()

    def setup_observations(self):
        #print("WARNING: setup_actions is not doing anything. Implement your own code in this method.")
        
        if Baseline:
            return self._g2op_env.observation_space
        
        obs_attr_to_keep = ["rho", "p_or", "gen_p", "load_p"]
        if "obs_attr_to_keep" in self.env_config:
            obs_attr_to_keep = copy.deepcopy(self.env_config["obs_attr_to_keep"])
        self._gym_env.observation_space.close()
        self._gym_env.observation_space = BoxGymObsSpace(self._g2op_env.observation_space,
                                                         attr_to_keep=obs_attr_to_keep
                                                         )
        # export observation space for the Grid2opEnv
        observation_space = Box(shape=self._gym_env.observation_space.shape,
                                     low=self._gym_env.observation_space.low,
                                     high=self._gym_env.observation_space.high)
        return observation_space

    def setup_actions(self):
        # TODO: Your code to specify & modify the action space goes here
        # See Grid2Op 'getting started' notebooks for guidance
        #  - Notebooks: https://github.com/rte-france/Grid2Op/tree/master/getting_started
        #print("WARNING: setup_actions is not doing anything. Implement your own code in this method.")
        
        act_type = "discrete"
        if "discrete" in self.env_config:
            act_type = self.env_config["act_type"]

        self._gym_env.action_space.close()
        if act_type == "discrete":
            # user wants a discrete action space
            act_attr_to_keep =  ["set_line_status","redispatch","set_bus"]
            if "act_attr_to_keep" in self.env_config:
                act_attr_to_keep = copy.deepcopy(self.env_config["act_attr_to_keep"])
            self._gym_env.action_space = DiscreteActSpace(self._g2op_env.action_space,
                                                          attr_to_keep=act_attr_to_keep)
            action_space = Discrete(self._gym_env.action_space.n)
        else:
            raise NotImplementedError(f"action type '{act_type}' is not currently supported.")
        
        return action_space 
    
    def reset(self, seed=None):
        return self._gym_env.reset(seed=seed, options=None)

    def step(self, action):
        return self._gym_env.step(action)

    def render(self):
        # TODO: Modify for your own required usage
        return self._gym_env.render()


## DQN agent interacting in environment

## Baseline Model

In [ ]:
config = {
    "policy_type": "MlpPolicy",
    "total_timesteps": 500000,
    "env_name": "l2rpn_case14_sandbox",
}
run = wandb.init(
    project="sb3",
    config=config,
    sync_tensorboard=True,  # auto-upload sb3's tensorboard metrics
    monitor_gym=True,  # auto-upload the videos of agents playing the game
    save_code=True,  # optional
)


def make_env():
    env = Gym2OpEnvWrapper(training=False)
    env = Monitor(env)  # record stats such as returns
    return env


env = DummyVecEnv([make_env])
# env = VecVideoRecorder(
#     env,
#     f"videos/{run.id}",
#     record_video_trigger=lambda x: x % 2000 == 0,
#     video_length=200,
# )
model_baseline =DQN(config["policy_type"], env, verbose=1, tensorboard_log=f"runs/{run.id}", target_update_interval=0, gradient_steps=1)
model_baseline.learn(
    total_timesteps=config["total_timesteps"],
    callback=WandbCallback(
        gradient_save_freq=1,
        model_save_path=f"DQN_Final/{run.id}",
        verbose=1,
    ),
)
run.finish()
env.close()

In [73]:
env = Gym2OpEnvWrapper(training=False)
print(evaluate_policy(model_baseline, env, n_eval_episodes=200, render=False))
env.close()

/opt/conda/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(165.94738498230774, 176.1150839629198)

## 1st Improvement(Reduced Observation Space)

In [62]:
config = {
    "policy_type": "MlpPolicy",
    "total_timesteps": 500000,
    "env_name": "l2rpn_case14_sandbox",
}
run = wandb.init(
    project="sb3",
    config=config,
    sync_tensorboard=True,  # auto-upload sb3's tensorboard metrics
    monitor_gym=True,  # auto-upload the videos of agents playing the game
    save_code=True,  # optional
)


def make_env():
    env = Gym2OpEnvWrapper(training=False, baseline=False)
    env = Monitor(env)  # record stats such as returns
    return env


env = DummyVecEnv([make_env])
# env = VecVideoRecorder(
#     env,
#     f"videos/{run.id}",
#     record_video_trigger=lambda x: x % 2000 == 0,
#     video_length=200,
# )
model_DQN =DQN(config["policy_type"], env, verbose=1, tensorboard_log=f"runs/{run.id}", target_update_interval=1, gradient_steps=1)
model_DQN.learn(
    total_timesteps=config["total_timesteps"],
    callback=WandbCallback(
        gradient_save_freq=1,
        model_save_path=f"DQN_Final/{run.id}",
        verbose=1,
    ),
)
run.finish()
env.close()

/opt/conda/lib/python3.10/site-packages/grid2op/gym_compat/discrete_gym_actspace.py:260: UserWarning: The class "DiscreteActSpace" should mainly be used to consider only discrete actions (eg. set_line_status, set_bus or change_bus). Though it is possible to use "redispatch" when building it, be aware that this continuous action will be treated as discrete by splitting it into bins. Consider using the "BoxGymActSpace" for these attributes.
  warnings.warn(
wandb: WARNING When using several event log directories, please call `wandb.tensorboard.patch(root_logdir="...")` before `wandb.init`


Using cuda device
Logging to runs/8ftqamwx/DQN_1
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 6.25     |
|    ep_rew_mean      | 0.58     |
|    exploration_rate | 0.991    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 26       |
|    time_elapsed     | 0        |
|    total_timesteps  | 25       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 5.38     |
|    ep_rew_mean      | 0.462    |
|    exploration_rate | 0.984    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 26       |
|    time_elapsed     | 1        |
|    total_timesteps  | 43       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.92     |
|    ep_rew_mean      | 0.422    |
|    exploration_rate | 0.978    |
| time

global_step,▁▁▂▂▂▂▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇██
rollout/ep_len_mean,█▃▃▆▅▄▄▅▇▁▃▃▃▆▄▅▃▃▂▄▅▃▅▅▃▃▄▄▃▃▂▅▅▅▄▃▄▅▂▅
rollout/ep_rew_mean,█▅▄▅▅▄▄▆▅▆▂▂▆▆▂▆▆▁▄▄▄▅▅▅▅▅▆▆▅▄▇▃▆▆▄▃▃▄▄▅
rollout/exploration_rate,█▆▅▄▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
time/fps,█▆▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
global_step,24990
rollout/ep_len_mean,3.8
rollout/ep_rew_mean,0.21998
rollout/exploration_rate,0.05
time/fps,21


### DQN evaluation

In [ ]:
env = Gym2OpEnvWrapper(training=False, baseline=False)
print(evaluate_policy(model_DQN, env, n_eval_episodes=200, render=False))
env.close()

## DQN Replay Buffer

In [ ]:

# env = Gym2OpEnv()
# goal_selection_strategy = "future" 
# model_DQN_replay_buffer = DQN("MlpPolicy", env, verbose=1,
#     replay_buffer_class=ReplayBuffer,
# )



# model_DQN_replay_buffer.learn(total_timesteps=10000)
#model_DQN.learn(total_timesteps=10000, log_interval=4)

#model_DQN.save("dqn_grid2op")
#del model # remove to demonstrate saving and loading
#model = DQN.load("dqn_grid2op")

In [ ]:
config = {
    "policy_type": "MlpPolicy",
    "total_timesteps": 500000,
    "env_name": "l2rpn_case14_sandbox",
}
run = wandb.init(
    project="sb3",
    config=config,
    sync_tensorboard=True,  # auto-upload sb3's tensorboard metrics
    monitor_gym=True,  # auto-upload the videos of agents playing the game
    save_code=True,  # optional
)


def make_env():
    env = Gym2OpEnvWrapper(training=False, baseline=False)
    env = Monitor(env)  # record stats such as returns
    return env


env = DummyVecEnv([make_env])
model_DQN_replay_buffer = DQN(config["policy_type"], env, verbose=1, tensorboard_log=f"runs/{run.id}",replay_buffer_class=ReplayBuffer)
model_DQN_replay_buffer.learn(
     total_timesteps=config["total_timesteps"],
     callback=WandbCallback(
         gradient_save_freq=1,
         model_save_path=f"DQN_Final/{run.id}",
         verbose=1,
     ),
 )
run.finish()
env.close()

In [ ]:
env = Gym2OpEnvWrapper(training=False, baseline=False)
print(evaluate_policy(model_DQN_replay_buffer, env, n_eval_episodes=200, render=False))
env.close()

## Tweaking Of Results with DQN Replay Buffer

In [61]:

config = {
    "policy_type": "MlpPolicy",
    "total_timesteps": 500000,
    "env_name": "l2rpn_case14_sandbox",
}
run = wandb.init(
    project="sb3",
    config=config,
    sync_tensorboard=True,  # auto-upload sb3's tensorboard metrics
    monitor_gym=True,  # auto-upload the videos of agents playing the game
    save_code=True,  # optional
)


def make_env():
    env = Gym2OpEnvWrapper(training=True, baseline=False, Falseother_rewards={"linesreconnected": LinesReconnectedReward,'linescapacity': LinesCapacityReward,
                                          'closetoOverflow':CloseToOverflowReward, 'alert':AlertReward})
    env = Monitor(env)  # record stats such as returns
    return env


env = DummyVecEnv([make_env])
model_replay_buffer_reward = DQN(config["policy_type"], env, verbose=1, tensorboard_log=f"runs/{run.id}", replay_buffer_class=ReplayBuffer)
model_replay_buffer_reward.learn(
     total_timesteps=config["total_timesteps"],
     callback=WandbCallback(
         gradient_save_freq=1,
         model_save_path=f"DQN_Final/{run.id}",
         verbose=1,
     ),
 )
run.finish()
env.close()

global_step,▁▁▂▂▂▂▂▂▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇███
rollout/ep_len_mean,▅▇█▅▄▂▂▄▄▄▄▃▄▃▃▂▃▄▃▃▂▄▄▄▅▆▄▅▆▆▁▅▆▅▃▂▃▄▄▅
rollout/ep_rew_mean,▆██▅▅▅▄▃▃▃▅▅▆▆▅▄▄▄▄▃▄▃▅▅▇▇▅▆▆▅▁▁▆▇▄▃▄▄▅▆
rollout/exploration_rate,██▇▇▇▆▅▅▄▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
time/fps,████▅▅▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
global_step,8814
rollout/ep_len_mean,3.97
rollout/ep_rew_mean,0.27466
rollout/exploration_rate,0.05
time/fps,21


/opt/conda/lib/python3.10/site-packages/grid2op/gym_compat/discrete_gym_actspace.py:260: UserWarning: The class "DiscreteActSpace" should mainly be used to consider only discrete actions (eg. set_line_status, set_bus or change_bus). Though it is possible to use "redispatch" when building it, be aware that this continuous action will be treated as discrete by splitting it into bins. Consider using the "BoxGymActSpace" for these attributes.
  warnings.warn(
Exception ignored in: <function BaseEnv.__del__ at 0x7a364e7af370>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/grid2op/Environment/baseEnv.py", line 4289, in __del__
    self.close()
  File "/opt/conda/lib/python3.10/site-packages/grid2op/Environment/baseEnv.py", line 3629, in close
    self._reward_helper.close()
  File "/opt/conda/lib/python3.10/site-packages/grid2op/Reward/rewardHelper.py", line 144, in close
    self.template_reward.close()
  File "/opt/conda/lib/python3.10/site-packages/grid

Using cuda device
Logging to runs/onmrg41a/DQN_1
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.5      |
|    ep_rew_mean      | 0.594    |
|    exploration_rate | 0.993    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 20       |
|    time_elapsed     | 0        |
|    total_timesteps  | 18       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.38     |
|    ep_rew_mean      | 0.508    |
|    exploration_rate | 0.987    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 21       |
|    time_elapsed     | 1        |
|    total_timesteps  | 35       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.08     |
|    ep_rew_mean      | 0.418    |
|    exploration_rate | 0.981    |
| time

global_step,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇█████
rollout/ep_len_mean,▇▇▇▅▄▆▆▃▅▃▇▃▁▆▃▁▃▃▃▄▄▂▄▃▃▄▄▅▃▆▄▆▅▅▅▆▆▅▄█
rollout/ep_rew_mean,█▃▃▃▃▃▂▃▅▅▂▃▂▂▅▂▂▂▂▃▂▂▄▃▂▂▂▂▂▁▁▁▁▂▂▃▃▂▃▁
rollout/exploration_rate,█▆▆▄▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
time/fps,██▁▁▁██▁▁█▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
global_step,24996
rollout/ep_len_mean,3.32
rollout/ep_rew_mean,0.08268
rollout/exploration_rate,0.05
time/fps,21


In [ ]:
env = Gym2OpEnvWrapper(training=False, baseline=False)
print(evaluate_policy(model_replay_buffer_reward, env, n_eval_episodes=200, render=False))
env.close()

## DQN Target Network without tweaked rewards

In [82]:
from grid2op.Chronics import MultifolderWithCache  # highly recommended for training

config = {
    "policy_type": "MlpPolicy",
    "total_timesteps": 500000,
    "env_name": "l2rpn_case14_sandbox",
}
run = wandb.init(
    project="sb3",
    config=config,
    sync_tensorboard=True,  # auto-upload sb3's tensorboard metrics
    monitor_gym=True,  # auto-upload the videos of agents playing the game
    save_code=True,  # optional
)


def make_env():
    env = Gym2OpEnvWrapper(training=False, baseline=False,other_rewards={"linesreconnected": LinesReconnectedReward,'linescapacity': LinesCapacityReward,
                                          'closetoOverflow':CloseToOverflowReward, 'alert':AlertReward}, training=False)
    env = Monitor(env)  # record stats such as returns
    return env


env = DummyVecEnv([make_env])
model_DQN_Target = DQN(config["policy_type"], env, verbose=1, tensorboard_log=f"runs/{run.id}",
    batch_size=256,
    tau=1.0,
    gamma=0.99,
    train_freq=4,
    gradient_steps=1,
    target_update_interval=1000,
    exploration_fraction=0.2,
    exploration_initial_eps=1.0,
    exploration_final_eps=0.05,
    max_grad_norm=10)

model_DQN_Target.learn(
    total_timesteps=config["total_timesteps"],
    callback=WandbCallback(
        gradient_save_freq=1,
        model_save_path=f"DQN_Final/{run.id}",
        verbose=1,
    ),
)
run.finish()
env.close()

/opt/conda/lib/python3.10/site-packages/grid2op/gym_compat/discrete_gym_actspace.py:260: UserWarning: The class "DiscreteActSpace" should mainly be used to consider only discrete actions (eg. set_line_status, set_bus or change_bus). Though it is possible to use "redispatch" when building it, be aware that this continuous action will be treated as discrete by splitting it into bins. Consider using the "BoxGymActSpace" for these attributes.
  warnings.warn(
wandb: WARNING When using several event log directories, please call `wandb.tensorboard.patch(root_logdir="...")` before `wandb.init`


Using cuda device
Logging to runs/loe4jodf/DQN_1
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.25     |
|    ep_rew_mean      | -0.00111 |
|    exploration_rate | 0.998    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 16       |
|    time_elapsed     | 0        |
|    total_timesteps  | 13       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.75     |
|    ep_rew_mean      | 0.186    |
|    exploration_rate | 0.994    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 19       |
|    time_elapsed     | 1        |
|    total_timesteps  | 30       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.42     |
|    ep_rew_mean      | 0.144    |
|    exploration_rate | 0.992    |
| time

global_step,▁▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▇▇▇▇▇▇██
rollout/ep_len_mean,▂▅▆▄▁▅▅▅▄▆▇▆▄▆▄▂▃▅▅▄▃▆▃▆▇▅▅▆█▄▆▇▇▇▆▆▇▅▃▅
rollout/ep_rew_mean,▇▂▁▂▆▆▇▆▇▇▃▂▃▆▇▆▅▅▅▅▇▆▆▅▄▅▇█▇█▅▆▂▄▃▃▄▅▅▇
rollout/exploration_rate,█▇▆▆▆▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
time/fps,▅▅▅▁▁▅▅▅▅███▅▅▅▅▅▅▅▅▅▅▅▅▅█▅█████████████
global_step,24980
rollout/ep_len_mean,4.23
rollout/ep_rew_mean,0.32702
rollout/exploration_rate,0.05
time/fps,23


In [85]:
env = Gym2OpEnvWrapper(training=False, baseline=False)
print(evaluate_policy(model_DQN_Target, env, n_eval_episodes=200, render=False))
env.close()

/opt/conda/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(179.51918541821652, 207.05691080216232)